# How to start

As first step, start grass7 with:
    
    $ grass70 gis/nc_basic/user1
    
Then from the GRASS shell, change the directory, with:
    
    > cd doc/pygrass
    
and then run:
    
    > ipython notebook --pylab inline

# Modules

The Modules module permits to work directly with the GRASS GIS module

In [ ]:
from __future__ import print_function
from IPython.display import Image
from pprint import pprint
from subprocess import PIPE

import numpy as np

from show import show

print('Hello!')


## Replace BASH

GRASS6 and GRASS7, the traditional way for scripting.

Import grass script module, which has some useful functions:

In [ ]:
from grass import script
import subprocess

Let's have a look which function containing the word command are available, with:

We have seen:

* how we can replace Bash script with Python;
* how we can update existing Python script to use pygrass;
* play with the grass modules as objects, changing only few parameters;
* give more power to the user with the special parameters "*_".

# Exercise

## Time for coding!

Transform the following Bash script:

into a Python one:

In [ ]:
!g.region -p

Read the output of g.region and return a dictionary.

# Create a GRASS module

In [ ]:
print(slp.flags.overwrite) # --overwrite, --verbose, --quite

In [ ]:
slp.flags.overwrite = True

# run the module
slp.run()

### Methods

In [ ]:
print(slp)  # __str__

In [ ]:
repr(slp)  # __repr__

In [ ]:
from grass.pygrass.modules import Module

slp = Module("r.slope.aspect")

slp(elevation='elevation', 
    slope='slp',  aspect='asp', 
    format='percent', overwrite=True, 
    verbose=False, flags='a', run_=False)

print(slp.get_bash())
print(slp.get_python())

## Run as a function

In [ ]:
slp(elevation='elevation', slope='slp',  aspect='asp', format='percent', overwrite=True)

## Special parameters

### "run_" to run later

In [ ]:
slp(elevation='elevation', slope='slp',  aspect='asp', format='percent', overwrite=True, run_=False)

### "finish_" to manage the process

In [ ]:
from grass.pygrass.utils import looking

looking(script, '*command*')

In [ ]:
script.list_pairs(type='raster')

### Start and wait until the end of the process

In [ ]:
script.run_command('r.info', map='elevation')

### Parse the kwargs and return a list of parameters

In [ ]:
slp.inputs.format = 'percents'

In [ ]:
slp.inputs.format = 'percent'

In [ ]:
slp.outputs.slope = 'slope'

In [ ]:
slp.flags.g = True

In [ ]:
slp.flags.a = True

In [ ]:
show('slope')

Some small tips of the new interface in python/ipython:

* Tab inside brackets gives the help of the command;
* or using:

In [ ]:
r.slope_aspect?

Do something with the vectors:

In [ ]:
script.list_pairs(type='vect')

In [ ]:
v.to_rast(input='hospitals', output='hospitals', type="point", use='cat', overwrite=True, quiet=True)

In [ ]:
script.list_pairs(type='rast')

## Backwards compatibility

If you have a python script and want to convert it to use the new python interface, you just need to transform:

In [ ]:
#from grass.script.core import run_command
from grass.pygrass.modules import Module as run_command

run_command("r.info", map='elevation')
#etc...

## Module as object

Import the Module class, and instantiate the command, the name of the GRASS module is required.

In [ ]:
script.make_command('r.info', map='elevation', flags='r')

### Start and return a `Popen` object

In [ ]:
script.start_command('r.info', map='elevation')

In [ ]:
process = script.start_command('r.info', map='elevation', flags='r', stdout=PIPE)

In [ ]:
stdout = process.stdout

In [ ]:
print([line.strip().split(b'=') for line in stdout])

### Parse the output of a command

In [ ]:
script.parse_command('r.info', map='elevation', flags='g', delimiter='=')

In [ ]:
script.parse_command('g.region', flags='p', delimiter=':')

### Catch the command output

In [ ]:
region = script.pipe_command('r.info', map='elevation', flags='r')
region

In [ ]:
stdout = ''.join(line for line in region.stdout)
print(stdout)

### Catch and return the stdout

In [ ]:
stdout = script.read_command('r.info', map='elevation', flags='r')
print(stdout)

In [ ]:
mrange = dict([line.split(b'=') for line in stdout.split(b'\n') if line != b''])
for k in mrange:
    mrange[k] = float(mrange[k])
mrange

### Use the pipe as input for another command

Write the rules that we want to use for the reclassification:

In [ ]:
keys = ['low', 'medium low', 'medium', 'medium high', 'high']
vals = np.linspace(mrange[b'min']-1, mrange[b'max']+1, num=len(keys)+1, endpoint=True)

rvals = [(int(vals[i-1])+1, int(vals[i]), i, keys[i-1]) for i in range(1, len(vals))]
rules = '\n'.join(['%3d thru %3d = %2d %s' % v for v in rvals])
print(rules)

Now we can pass as input for the r.reclass the rules using the function `write_command`:

In [ ]:
script.write_command('r.reclass', input='elevation', output='elev', rules='-', overwrite=True, stdin=rules.encode())

### Display the map inside the ipython notebook

Start a new virtual monitor that generate a file: `view.png` that we can display inside the ipython notebook.

In [ ]:
script.run_command('d.mon', start='png', output='view.png', overwrite=True)

In [ ]:
script.run_command('d.rast', map='elevation')
Image('view.png')

In [ ]:
script.run_command('d.rast', map='elev')
Image('view.png')

In [ ]:
script.run_command('d.mon', stop='png')

## Using the pygrass module interface

User who just wants to replace BASH with python has to import modules in the following manner:

In [ ]:
slp(elevation='elevation', slope='slp',  aspect='asp', format='percent', overwrite=True, run_=True, finish_=False)

In [ ]:
slp.popen.wait() # .kill()

### `stdin_`, `stdout_`, `stderr_`

In [ ]:
from grass.pygrass.modules.shortcuts import raster as r, vector as v, general as g, display as d
from grass.pygrass.modules import Module

And then call the grass modules

In [ ]:
g.region(raster='elevation', flags='p')

In [ ]:
print(g.region(flags='p', stdout_=PIPE).outputs.stdout.decode())

If the name of the grass module contains "." you should replace it with "_". See the example bellow:

In [ ]:
r.slope_aspect(elevation='elevation', slope='slope', aspect='aspect', overwrite=True)
r.mapcalc("slope_gt_10 = if(slope > 10, slope, null())", overwrite=True)
print(r.info(map='slope_gt_10', flags='r', stdout_=PIPE).outputs.stdout.decode())

In [ ]:
show('slope_gt_10', flags='n')

In [ ]:
from grass.pygrass.modules import Module

slp = Module("r.slope.aspect")

### Attributes

In [ ]:
slp.name

In [ ]:
slp.description

In [ ]:
slp.keywords

In [ ]:
slp.label

In [ ]:
# get the documentation 'of the module
slp?

The instantiated module splits the parameters in:

* inputs
* outputs

both parameters are an [OrderedDict](http://docs.python.org/2/library/collections.html#collections.OrderedDict) python object. Additionally, the 'required' attribute returns a list of the parameters that are required.

